In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pprint import pprint
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
import time

In [ ]:
client = MongoClient('localhost', 27017) # Подключаемся к клиенту
db = client['mail_db'] # Создаем БД
mess_mail = db.mess_mail  # Создаем объект, который создаст коллекцию mes_mail в БД mail_db

firefox_options = Options()
firefox_options.add_argument('start-maximized')


driver = webdriver.Firefox(executable_path='/home/vyacheslav/geckodriver', options=firefox_options)
driver.maximize_window()
driver.get('https://mail.ru/')

DLT = None

time.sleep(2)

login = driver.find_element_by_id('mailbox:login')
login.send_keys('study.ai_172@mail.ru')
login.send_keys(Keys.RETURN)

time.sleep(2)

password = driver.find_element_by_id('mailbox:password')
password.send_keys('NextPassword172')
password.send_keys(Keys.RETURN)

time.sleep(7)

mess_links = set()
last_mess = None


count = DLT
while True:

    if (count is not None) and (count <= 0): break

    time.sleep(2)
    messages = driver.find_elements_by_class_name('js-letter-list-item')

    for message in messages:
        mess_links.add(message.get_attribute('href'))
        if (count is not None):
            count -= 1
            if  count <= 0: break

    if len(messages) > 0 and messages[-1] != last_mess:
        last_mess = messages[-1]
        actions = ActionChains(driver)
        actions.move_to_element(last_mess)
        actions.perform()
    else:
        break

print('Messages: ', len(mess_links))


count_add = 0
count_s = 0
for link in mess_links:

    if mess_mail.count_documents({'_id': get_id_from_link(link)}) != 0:
        
        count_s += 1
        print(f'Letters skipped: {count_s}')
        continue

    driver.get(link)

    time.sleep(15)

    elem = driver.find_element_by_class_name('letter-contact')
    mail = elem.get_attribute('title')
    author = elem.text

    elem = driver.find_element_by_class_name('letter__date')
    date = elem.text

    elem = driver.find_element_by_class_name('thread__subject')
    head = elem.text

    elem = driver.find_element_by_class_name('letter__body')
    text = elem.text

    message_full = {'_id' : get_id_from_link(link),
                   'mail' : mail,
                   'author' : author,
                   'date' : date,
                   'head' : head,
                   'text' : text}


    
    mess_mail.insert_one(message_full)
    count_add += 1
    print(f'Messages added: {count_add}')


driver.quit()